In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Классификация запрещенных комментариев 

Этот ноутбук основан на одном из заданий [ШАД](https://github.com/yandexdataschool/nlp_course/blob/2023/week02_classification/homework_part1.ipynb)

![img](https://github.com/yandexdataschool/nlp_course/raw/master/resources/banhammer.jpg)

__В этом ноутбуке__ вы построите алгоритм который классифицирует комментарии на нормальные или токсичные.
Как и во многих реальных случаях, у вас есть только небольшой (10 ^ 3) набор данных с примерами, помеченными вручную, для работы. Мы решим эту проблему, используя классические методы nlp.

In [5]:
import pandas as pd
data = pd.read_csv("comments.tsv", sep='\t')

texts = data['comment_text'].values
target = data['should_ban'].values
data[50::200]

,should_ban,comment_text
50,0,"""Those who're in advantageous positions are th..."
250,1,Fartsalot56 says f**k you motherclucker!!
450,1,"Are you a fool? \n\nI am sorry, but you seem t..."
650,1,I AM NOT A VANDAL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
850,0,Citing sources\n\nCheck out the Wikipedia:Citi...


In [6]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(texts, target, test_size=0.5, random_state=42)

__ Примечание:__ как правило, хорошей идеей является разделение данных на обучающие / тестовые, прежде чем что-либо с ними делать.

Это защищает вас от возможной утечки данных на этапе предварительной обработки. Например, если вы решите выбрать слова, присутствующие в непристойных твитах, в качестве функций, вам следует учитывать только эти слова в обучающем наборе. В противном случае ваш алгоритм может обмануть оценку.

### Preprocessing and tokenization

Комментарии содержат необработанный текст с пунктуацией, заглавными/строчными буквами и даже символами новой строки.

Чтобы упростить все дальнейшие шаги, мы разделим текст на токены, разделенные пробелами, используя один из токенизаторов nltk.

In [7]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

text = 'How to be a grown-up at work: replace "shit" with "Ok, great!".'
print("before:", text,)
print("after:", preprocess(text),)

before: How to be a grown-up at work: replace "shit" with "Ok, great!".
after: how to be a grown-up at work : replace " shit " with " ok , great ! " .


In [8]:
# task: Preprocess каждый твит в обучающем и тестовом множестве

texts_train = [preprocess(text) for text in texts_train]
texts_test = [preprocess(text) for text in texts_test]

In [9]:
assert texts_train[5] ==  'who cares anymore . they attack with impunity .'
assert texts_test[89] == 'hey todds ! quick q ? why are you so gay'
assert len(texts_test) == len(y_test)

### Решаем: bag of words

![img](http://www.novuslight.com/uploads/n/BagofWords.jpg)

Одним из традиционных подходов к решению такой проблемы является использование фичей bag of words:
1. создайте словарь часто употребляемых слов (используйте только данные из обучения)
2. для каждой обучающей выборки подсчитайте, сколько раз в ней встречается слово (для каждого слова в словарном запасе).
3. рассматривайте этот подсчет как фичи для некоторого классификатора

__Note:__ in practice, you can compute such features using sklearn. Please don't do that in the current assignment, though.
* `from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer`

In [22]:
# task: Найдите до k самых часто встречающихся токенов в тексатх обучающей выборки
# Отсортируйте их по количеству вхождений
k = 5000

from collections import Counter
cnt = Counter()
for text in texts_train:
    cnt.update(text.split())

bow_vocabulary = [w[0] for w in cnt.most_common(k)]

print('example features:', sorted(bow_vocabulary)[::100])

example features: ['!', '14:54', '430', 'accidental', "ain't", 'apologise', 'attention', 'begs', 'botticelli', 'care', 'civil', 'concerning', 'cossacks', 'day', 'determined', 'dogs', 'eh', 'every', 'fatuorum', 'forgotten', 'german', 'hahahahahaha', 'hist', 'imperial', 'intervening', 'justification', 'least', 'look', "mclaren's", 'moore', 'neva', 'obvious', 'pair', 'pieces', 'preconceptions', 'puff', 'recliner', 'researchers', 'rumours', 'sending', 'sincerly', 'spam', 'struggle', 'tagged', 'thing', 'treasury', 'unless', 'visited', 'whomever', 'xizer']


In [17]:
def text_to_bow(text):
    """ convert text string to an array of token counts. Use bow_vocabulary. """
    ans = np.zeros(k)
    for w in text.split():
        if (w in bow_vocabulary):
            ans[bow_vocabulary.index(w)] += 1
    return np.array(ans, 'float32')

In [18]:
X_train_bow = np.stack(list(map(text_to_bow, texts_train)))
X_test_bow = np.stack(list(map(text_to_bow, texts_test)))

In [19]:
k_max = len(set(' '.join(texts_train).split()))
assert X_train_bow.shape == (len(texts_train), min(k, k_max))
assert X_test_bow.shape == (len(texts_test), min(k, k_max))
assert np.all(X_train_bow[5:10].sum(-1) == np.array([len(s.split()) for s in  texts_train[5:10]]))
assert len(bow_vocabulary) <= min(k, k_max)
assert X_train_bow[6, bow_vocabulary.index('.')] == texts_train[6].split().count('.')

__Naive bayes:__ возможно, самой простой моделью, которая может решить вашу проблему, является так называемый наивный байесовский классификатор. 
Это тривиальная линейная модель, которая предполагает независимость входных фичей и вычисляет коэффициенты путем, ???, подсчета вероятностей.

In [ ]:
class BinaryNaiveBayes:
    delta = 1.0  # добавляется чтобы сгладить вероятности
    
    def fit(self, X, y):
        """
        Fit a NaiveBayes classifier for two classes
        :param X: [batch_size, vocab_size] of bag-of-words features
        :param y: [batch_size] of binary targets {0, 1}
        """
        # посчитайте обычные вероятности классов, p(y=k) for k = 0,1
        self.p_y = np.array(<YOUR CODE: probability of y=0 and of y=1 in that order>)
        
        # посчитайте количество вхождений каждого слова в тексты с метками 0 и 1 раздельно
        word_counts_positive = <YOUR CODE HERE>
        word_counts_negative = <YOUR CODE HERE>
        # должны быть размера [vocab_size].
        
        # Наконец, используйте эти вектора чтобы оценить p(x | y = k) для k = 0, 1
        
        <YOUR CODE HERE>
        self.p_x_given_positive = <...>
        self.p_x_given_negative = <...>
        # должен быть размера [vocab_size]; не забудьте добавить self.delta!
        
        return self
    
    def predict_scores(self, X):
        """
        :param X: [batch_size, vocab_size] of bag-of-words features
        :returns: a matrix of scores [batch_size, k] of scores for k-th class
        """
        # compute scores for positive and negative classes separately.
        # these scores should be proportional to log-probabilities of the respective target {0, 1}
        # note: if you apply logarithm to p_x_given_*, the total log-probability can be written
        # as a dot-product with X
        score_negative = X.dot(np.log(self.p_x_given_negative))
        score_positive = X.dot(np.log(self.p_x_given_positive))
        
        # you can compute total p(x | y=k) with a dot product
        return np.stack([score_negative, score_positive], axis=-1)
    
    def predict(self, X):
        return self.predict_scores(X).argmax(axis=-1)

In [ ]:
naive_model = BinaryNaiveBayes().fit(X_train_bow, y_train)

In [ ]:
assert naive_model.p_y.shape == (2,) and naive_model.p_y.sum() == 1 and naive_model.p_y[0] > naive_model.p_y[1]
assert naive_model.p_x_given_positive.shape == naive_model.p_x_given_negative.shape == X_train_bow.shape[1:]
assert np.allclose(naive_model.p_x_given_positive.sum(), 1.0)
assert np.allclose(naive_model.p_x_given_negative.sum(), 1.0)
assert naive_model.p_x_given_negative.min() > 0, "did you forget to add delta?"

f_index = bow_vocabulary.index('fuck')  # offensive tweets should contain more of this
assert naive_model.p_x_given_positive[f_index] > naive_model.p_x_given_negative[f_index]

g_index = bow_vocabulary.index('good')  # offensive tweets should contain less of this
assert naive_model.p_x_given_positive[g_index] < naive_model.p_x_given_negative[g_index]

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

for name, X, y, model in [
    ('train', X_train_bow, y_train, naive_model),
    ('test ', X_test_bow, y_test, naive_model)
]:
    proba = model.predict_scores(X)[:, 1] - model.predict_scores(X)[:, 0]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

test_accuracy = np.mean(naive_model.predict(X_test_bow) == y_test)
print(f"Model accuracy: {test_accuracy:.3f}")
assert test_accuracy > 0.75, "Accuracy too low. There's likely a mistake in the code."
print("Well done!")

Ладно, он определенно научился *чему-то*. Теперь давайте разберемся, чему именно он научился. Самый простой способ сделать это - выделить, какие слова имеют наибольшее соотношение положительной и отрицательной вероятности или наоборот. Мы остановимся на первом.

__Ваша задача__ это подсчитать top-25 слов у которых __наибольшее__ соотношение ${p(x_i | y=1)} \over {p(x_i | y=0)}$.

In [ ]:
# hint: use naive_model.p_*
probability_ratio = <YOUR CODE: compute the ratio as defined above, must be a vector of [vocab_size]>
top_negative_words = <YOUR CODE: find 25 words with highest probability_ratio, return list of str>

assert len(top_negative_words) == 25 and [isinstance(w, str) for w in top_negative_words]
assert 'j.delanoy' in top_negative_words and 'college' in top_negative_words

for i, word in enumerate(top_negative_words):
    print(f"#{i}\t{word.rjust(10, ' ')}\t(ratio={probability_ratio[bow_vocabulary.index(word)]})")

Теперь давайте попробуем что-нибудь менее доисторическое: __Логистическая регрессия__. Вы можете найти веса модели, оптимизировав логарифмическую вероятность ответа. Хотя, конечно, вам даже не нужно больше писать это от руки. Давайте обучим это!

In [ ]:
from sklearn.linear_model import LogisticRegression
bow_model = <YOUR CODE HERE - train a logistic regression>

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

for name, X, y, model in [
    ('train', X_train_bow, y_train, bow_model),
    ('test ', X_test_bow, y_test, bow_model)
]:
    proba = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

test_accuracy = np.mean(bow_model.predict(X_test_bow) == y_test)
print(f"Model accuracy: {test_accuracy:.3f}")
assert test_accuracy > 0.77, "Hint: tune the parameter C to improve performance"
print("Well done!")

### Task: implement TF-IDF features

Not all words are equally useful. One can prioritize rare words and downscale words like "and"/"or" by using __tf-idf features__. This abbreviation stands for __text frequency/inverse document frequence__ and means exactly that:

$$ feature_i = { Count(word_i \in x) \times { log {N \over Count(word_i \in D) + \alpha} }} $$


, where x is a single text, D is your dataset (a collection of texts), N is a total number of documents and $\alpha$ is a smoothing hyperparameter (typically 1). 
And $Count(word_i \in D)$ is the number of documents where $word_i$ appears.

It may also be a good idea to normalize each data sample after computing tf-idf features.

__Your task:__ implement tf-idf features, train a model and evaluate ROC curve. Compare it with basic BagOfWords model from above.

Please don't use sklearn/nltk builtin tf-idf vectorizers in your solution :) You can still use 'em for debugging though.